# 13-Testing

In [3]:
from scipy import *

In [4]:
def bisect(f, a, b, tol=1.e-8):
    """
    Implementation of the bisection algorithm 
    f real valued function
    a,b interval boundaries (float) with the property 
    f(a) * f(b) <= 0
    tol tolerance (float)
    """
    if f(a) * f(b)> 0:
        raise ValueError("Incorrect initial interval [a, b]") 
    for i in range(100):
        c = (a + b) / 2.
        if f(a) * f(c) <= 0:
            b = c
        else:
            a = c
        if abs(a - b) < tol:
            return (a + b) / 2
    raise Exception(
          'No root found within the given tolerance {}'.format(tol))

Test to see that is finds the root of $f(x)=x$

In [5]:
def test_identity():
    result = bisect(lambda x: x, -1., 1.) 
    expected = 0.
    assert allclose(result, expected),'expected zero not found'

test_identity()

Test to see if the sign condition $f(a)f(b)< 0$ is checked

In [6]:
def test_badinput():
    try:
        bisect(lambda x: x,0.5,1)
    except ValueError:
        pass
    else:
        raise AssertionError()

test_badinput()

In [7]:
def test_equal_boundaries():
    result = bisect(lambda x: x, 0., 0.)
    expected = 0.
    assert allclose(result, expected), \
                    'test equal interval bounds failed'

In [8]:
test_equal_boundaries()

In [9]:
def test_reverse_boundaries():
    result = bisect(lambda x: x, 1., -1.)
    expected = 0.
    assert allclose(result, expected),\
                  'test reverse interval bounds failed'

In [10]:
test_reverse_boundaries()

In [15]:
run TestIdentity

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [16]:
run Testing_all

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [17]:
run TestString

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [18]:
run Tests_several

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [19]:
import scipy.linalg as sl
A=rand(10,10)
[Q,R]=sl.qr(A)

In [22]:
import numpy.testing as npt 
npt.assert_allclose(dot(Q.T,Q),identity(Q.shape[0]),atol=1.e-12)

In [24]:
npt.assert_allclose(dot(Q,R),A)

In [26]:
run TestQR

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [27]:
test_case = TestIdentity(methodName='test_functionality')

In [28]:
test_case.debug()

In [30]:
A=zeros((1000,1000))
A[53,67]=10

def find_elements_1(A):
    b = []
    n, m = A.shape
    for i in range(n):
        for j in range(m):
            if abs(A[i, j]) > 1.e-10:
                b.append(A[i, j])
    return b

def find_elements_2(A):
    return [a for a in A.reshape((-1, )) if abs(a) > 1.e-10]

def find_elements_3(A):
    return [a for a in A.flatten() if abs(a) > 1.e-10]
 
def find_elements_4(A):
    return A[where(0.0 != A)]

In [32]:
%timeit -n 50 -r 3 find_elements_1(A)

50 loops, best of 3: 334 ms per loop


In [33]:
%timeit -n 50 -r 3 find_elements_2(A)

50 loops, best of 3: 174 ms per loop


In [34]:
%timeit -n 50 -r 3 find_elements_3(A)

50 loops, best of 3: 175 ms per loop


In [35]:
%timeit -n 50 -r 3 find_elements_4(A)

50 loops, best of 3: 3.79 ms per loop


In [36]:
import timeit
setup_statements="""
from scipy import zeros
from numpy import where
A=zeros((1000,1000))
A[57,63]=10.

def find_elements_1(A):
    b = []
    n, m = A.shape
    for i in range(n):
        for j in range(m):
            if abs(A[i, j]) > 1.e-10:
               b.append(A[i, j])
    return b

def find_elements_2(A):
    return [a for a in A.reshape((-1,)) if abs(a) > 1.e-10]

def find_elements_3(A):
    return [a for a in A.flatten() if abs(a) > 1.e-10]

def find_elements_4(A):
    return A[where( 0.0 != A)]
"""
experiment_1 = timeit.Timer(stmt = 'find_elements_1(A)',
                            setup = setup_statements)
experiment_2 = timeit.Timer(stmt = 'find_elements_2(A)',
                            setup = setup_statements)
experiment_3 = timeit.Timer(stmt = 'find_elements_3(A)',
                            setup = setup_statements)
experiment_4 = timeit.Timer(stmt = 'find_elements_4(A)',
                            setup = setup_statements)

In [38]:
t1 = experiment_1.repeat(3,5) 
t2 = experiment_2.repeat(3,5) 
t3 = experiment_3.repeat(3,5) 
t4 = experiment_4.repeat(3,5) 
# Results per loop in ms
print(min(t1)*1000/5) # 615 ms
print(min(t2)*1000/5) # 543 ms
print(min(t3)*1000/5) # 546 ms
print(min(t4)*1000/5) # 7.26 ms

335.6477557972539
182.16045520093758
185.94143340014853
3.827910599648021


In [39]:
import time
class Timer:
	def __enter__(self):
		self.start = time.time()
	def __exit__(self, ty, val, tb):
		end = time.time()
		self.elapsed=end-self.start
		print('Time elapsed {} seconds'.format(self.elapsed))
		return False 

In [42]:
with Timer():
    find_elements_1(A)

Time elapsed 0.3459136486053467 seconds
